<a href="https://colab.research.google.com/github/Flavio-AzL/challenge-alura-store/blob/main/AluraStoreBr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importação dos dados



In [2]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_1.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_4.csv"

loja1 = pd.read_csv(url1)
loja2 = pd.read_csv(url2)
loja3 = pd.read_csv(url3)
loja4 = pd.read_csv(url4)

loja1.head()

,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,lat,lon
0,Assistente virtual,eletronicos,219.08,9.249790,16/01/2021,Pedro Gomes,SP,4,cartao_credito,8,-22.19,-48.79
1,Mesa de jantar,moveis,256.35,11.234305,18/05/2022,Beatriz Moraes,RJ,1,cartao_credito,4,-22.25,-42.66
2,Jogo de tabuleiro,brinquedos,279.51,21.262681,15/03/2021,João Souza,DF,1,cartao_credito,1,-15.83,-47.86
3,Micro-ondas,eletrodomesticos,1009.99,54.667344,03/05/2022,João Souza,RS,4,boleto,1,-30.17,-53.50
4,Cadeira de escritório,moveis,446.99,26.964689,07/11/2020,Larissa Alves,MG,5,boleto,1,-18.10,-44.38


## *00. Explorações iniciais dos Datasets*

In [3]:
print(loja1.shape)
print(loja2.shape)
print(loja3.shape)
print(loja4.shape)

# Observado dif. na quantidade de linhas para Loja4


(2359, 12)
(2359, 12)
(2359, 12)
(2358, 12)


In [4]:
print(loja1.columns)
print(loja2.columns)
print(loja3.columns)
print(loja4.columns)

Index(['Produto', 'Categoria do Produto', 'Preço', 'Frete', 'Data da Compra',
       'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'lat', 'lon'],
      dtype='object')
Index(['Produto', 'Categoria do Produto', 'Preço', 'Frete', 'Data da Compra',
       'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'lat', 'lon'],
      dtype='object')
Index(['Produto', 'Categoria do Produto', 'Preço', 'Frete', 'Data da Compra',
       'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'lat', 'lon'],
      dtype='object')
Index(['Produto', 'Categoria do Produto', 'Preço', 'Frete', 'Data da Compra',
       'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'lat', 'lon'],
      dtype='object')


#1. Análise do faturamento


In [9]:
# Organizando os dataframes para facilitar a visualzação com rótulos 'Loja x':
lojas = [loja1, loja2, loja3, loja4]
nomes = ['Loja 1', 'Loja 2', 'Loja 3', 'Loja 4']

# Análise de Faturamento de Vendas, por loja:
for loja, preco in zip(nomes, lojas):
  vendas = preco['Preço'].sum()
  print(f'{loja}: R$ {vendas:.2f}')


Loja 1: R$ 1534509.12
Loja 2: R$ 1488459.06
Loja 3: R$ 1464025.03
Loja 4: R$ 1384497.58


In [ ]:
# Análise do valor de Fretes recebidos por loja:
for loja, frete in zip(nomes, lojas):
  fretes = frete['Frete'].sum()
  print(f'{loja}: R$ {fretes:.2f}')


Loja 1: R$ 81837.97
Loja 2: R$ 79314.16
Loja 3: R$ 78022.66
Loja 4: R$ 73755.88


In [51]:
# Análise de Vendas por Ano
for loja in lojas:
    loja['Data da Compra'] = pd.to_datetime(loja['Data da Compra'], dayfirst=True)
    loja['Ano'] = loja['Data da Compra'].dt.year


# Adiciona a coluna 'Loja' dinamicamente em cada DataFrame
for loja_df, nome in zip(lojas, nomes):
    loja_df['Loja'] = nome

# Junta todos os DataFrames em um só
todas_lojas = pd.concat(lojas, ignore_index=True)
faturamento_total = round(todas_lojas.groupby('Loja')[['Preço', 'Frete']].sum(), 2)
faturamento_total['Total'] = round(faturamento_total['Preço'] - faturamento_total['Frete'], 2)
faturamento_total

todas_lojas['Faturamento Líquido'] = todas_lojas['Preço'] - todas_lojas['Frete']

faturamento_pivot = round(faturamento_ano.pivot(index='Ano', columns='Loja', values='Faturamento Líquido'), 2)
faturamento_pivot


Loja,Loja 1,Loja 2,Loja 3,Loja 4
Ano,,,,
2020,465583.43,404397.85,406186.14,417757.05
2021,457203.49,443464.53,458091.57,438966.69
2022,399602.68,452175.31,442280.01,381420.18
2023,130281.55,109107.21,79444.65,72597.78


In [49]:
crescimento_percentual = faturamento_pivot.pct_change().round(4) * 100
print(crescimento_percentual)
print('\n')

media_crescimento = crescimento_percentual.mean().round(2)
print(media_crescimento)



Loja  Loja 1  Loja 2  Loja 3  Loja 4
Ano                                 
2020     NaN     NaN     NaN     NaN
2021    -1.8    9.66   12.78    5.08
2022   -12.6    1.96   -3.45  -13.11
2023   -67.4  -75.87  -82.04  -80.97


Loja
Loja 1   -27.27
Loja 2   -21.42
Loja 3   -24.24
Loja 4   -29.67
dtype: float64


# 2. Vendas por Categoria


In [52]:
pd.pivot_table(
    todas_lojas,
    values='Preço',
    index='Categoria do Produto',
    columns='Loja',
    aggfunc='sum',
    fill_value=0
).round(2)


Loja,Loja 1,Loja 2,Loja 3,Loja 4
Categoria do Produto,,,,
brinquedos,23993.78,21262.21,25869.20,27015.67
eletrodomesticos,484913.36,464758.13,438984.89,377680.65
eletronicos,572659.23,547773.71,547699.92,545966.86
esporte e lazer,52387.55,46326.09,47456.10,44465.65
instrumentos musicais,121731.69,139987.03,103174.45,100136.50
livros,11713.92,13454.35,12664.70,12427.77
moveis,250178.11,235234.68,268095.56,256705.65
utilidades domesticas,16931.48,19662.86,20080.21,20098.83


# 3. Média de Avaliação das Lojas

# 4. Produtos Mais e Menos Vendidos

# 5. Frete Médio por Loja
